# Projeto 1 - Ciência dos Dados

Nome: Marcelo da Costa Poltronieri

Nome: Pedro Nery Affonso dos Santos

Nome: Victor de Almeida Cunha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [133]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [134]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Victor Cunha\Desktop\Atividades\C-Dados\23-1b-cd-p1-grupo_marcelo\notebooks


Carregando a base de dados com os tweets classificados manualmente:

In [135]:
filename = '../data/dados.xlsx'

In [136]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Avaliacao,Target
0,"A insatisfação com o trabalho, a importância d...",1
1,"Nossa, que livro ruim. Muito mais muito prolix...",0
2,"Não vale a pena, é uma ofensa aos Potterfanati...",0
3,O mais impressionante é que os personagens do ...,1
4,Olavo será um dos responsáveis pela destruição...,2


In [137]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Avaliacao,Target
0,Pense naquele livro que no caso de estar a pro...,1
1,"Muito bom o livro e recomendo, é um verdadeiro...",1
2,"O livro cumpre muito bem sua proposta, gera bo...",1
3,Este livro é um clássico. Simplesmente maravil...,1
4,Este clássico é mágico e fascinante em seus en...,1


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

O assunto escolhido pelo grupo foi o de livros vendidos pela plataforma da Amazon. Para que as avaliações possam realmente refletir o conteúdo do livro, ou seja, o sentimento popular sobre a obra lida, foram escolhidos três critérios: comentários que elaboram críticas **positivas** ao **conteúdo** do livro (1), comentários **negativos** em relação ao **conteúdo** do livro (0) e, por último, comentários **não** referentes ao **conteúdo do livro ou resumos da obra**, como, por exemplo, reclamações sobre a entrega ou sobre traduções e edições. (2)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

**Breve explicação do Classificador Naive-Bayes:**

O objetivo do projeto consiste em montar um classificador probabilístico capaz de escolher corretamente, dada uma certa frase, entre as três opções de variável Target expostos anteriormente (1, 2, ou 0). Ou seja, aquilo que se deseja que o bot faça é calcular, para cada comentário contido na planilha de Excel, as seguintes probabilidades condicionais: **P(0 | frase)**, **P(1 | frase)** e **P(2 | frase)**. 

Para calcular tais probabilidades, o robô será programado de tal forma que seja capaz de analisar as **frequências de cada palavra na frase** e escolher entre a maior das três. Ao final da frase, deseja-se que o bot tome uma decisão: afinal, aquela frase tem maior probabilidade de ser uma análise crítica positiva(1), negativa (0), ou nem se refere ao conteúdo do livro (2)?

Após a montagem do robô, deseja-se descobrir, por meio do comando pd.crosstab(), a taxa de acertos do robô. Em relação àquilo que era esperado dele, qual foi a eficiência do código quando se deparou com frases classificadas como 1, 0 e 2?

**A seguir, é possível visualizar a montagem do Classificador Naive-Bayes em Python:**

In [138]:
import re

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


Primeiramente, deseja-se fornecer ao classificador as frequências relativas de cada palavra em frases que sejam positivas, negativas ou irrelevantes. Assim, dado que já foi feita uma classificação manual pelo grupo na planilha de Excel, faz-se necessário realizar quatro filtros das frases (dado que também será necessário mais tarde treinar o robô, sem acesso às respostas) por meio do comando **.loc[]** e guardá-los em suas respectivas variáveis.

Com os filtros prontos e guardados, coloca-se todas as frases em seguida em uma lista por meio do comando **.values.tolist()**.

In [139]:
# Separando as avaliações positivas, negativas, irrelevantes e o total em listas

positivo = train.loc[train['Target']==1,['Avaliacao']]
negativo = train.loc[train['Target']==0,['Avaliacao']]
irrelevante = train.loc[train['Target']==2,['Avaliacao']]
treinamento = train.loc[:,['Avaliacao']]

positivo_list = positivo['Avaliacao'].values.tolist()
negativos_list = negativo['Avaliacao'].values.tolist()
irrelevantes_list = irrelevante['Avaliacao'].values.tolist()
treinamento_list = treinamento['Avaliacao'].values.tolist()

Agora que já temos as frases devidamente divididas e guardadas em listas com suas respectivas classificações, precisamos fazer um **.split()** para obter as palavras de cada frase. Entretanto, antes disso, como muitos dos comentários na base de dados apresentam elementos que podem atrapalhar na análise do bot (pontos de interrogação, exclamação, vírgulas, etc), devemos retirá-los. Para isso, usaremos um loop em **while** e a **função cleanup** definida anteriormente para guardar as frases, agora limpas, em novas listas. 

In [140]:
# Limpando todas as frases:

r = 0
positivos_list_clean = []

ir = 0
irrelevantes_list_clean = []

n = 0
negativos_list_clean = []

t = 0
treinamento_list_clean = []

while r<len(positivo_list):
    positivos_list_clean.append(cleanup(positivo_list[r]))
    r += 1

while ir<len(irrelevantes_list):
    irrelevantes_list_clean.append(cleanup(irrelevantes_list[ir]))
    ir += 1

while n<len(negativos_list):
    negativos_list_clean.append(cleanup(negativos_list[n]))
    n += 1

while t<len(treinamento_list):
    treinamento_list_clean.append(cleanup(treinamento_list[t]))
    t += 1

Finalmente, por meio de um laço **for**, faz-se a separação de palavra em palavra para cada uma das frases das listas de comentários positivios, negativos, irrelevantes e de treinamento. Cria-se, em seguida, um Data Frame contendo todas as palavras nas listas. O Data Frame nos interessa, pois ele irá fornecer ao bot as frequências relativas de cada palavra por meio do comando **.value_counts(normalize=True)**.

In [141]:
# Separando as frases em palavras

pos_palavras = []
irrel_palavras = []
neg_palavras = []
todas_palavras = []

for frase in positivos_list_clean:
    palavras = frase.split()
    for palavra in palavras:
        pos_palavras.append(palavra)

# Após a separação, cria-se um Dataframe com todas as palavras de tal divisão

dfpos = pd.Series(pos_palavras)

for frase in irrelevantes_list_clean:
    palavras = frase.split()
    for palavra in palavras:
        irrel_palavras.append(palavra)

dfirrel = pd.Series(irrel_palavras)

for frase in negativos_list_clean:
    palavras = frase.split()
    for palavra in palavras:
        neg_palavras.append(palavra)

dfneg = pd.Series(neg_palavras)

for frase in treinamento_list_clean:
    palavras = frase.split()
    for palavra in palavras:
        todas_palavras.append(palavra)

dfall = pd.Series(todas_palavras)

Com os Data Frames obtidos, é possível descobrir por meio dos seus tamanhos as probabilidade de uma palavra ser positiva, irrelevante ou negativa, já que temos o espaço amostral no DF contendo todas as palavras.

In [142]:
# Análise da probabilidade de uma palavra ser positiva, negativa ou irrelevante

P = len(dfpos)/len(dfall)
IR = len(dfirrel)/len(dfall)
N = len(dfneg)/len(dfall)

print(f"Probabilidade de ser uma palavra positiva: {P}")
print(f"Probabilidade de ser uma palavra irrelevante: {IR}")
print(f"Probabilidade de ser uma palavra negativa: {N}")

Probabilidade de ser uma palavra positiva: 0.44265075962345046
Probabilidade de ser uma palavra irrelevante: 0.26170192935035885
Probabilidade de ser uma palavra negativa: 0.2956473110261907


Agora, a fim de fazer o bot classificar, será necessário usar a **Suavização de LaPlace**.

A Suavização de LaPlace é uma técnica utilizada para lidar com o problema de frequências zero em um conjunto de dados. 
Ela é aplicada adicionando-se um valor constante a todas as contagens de cada categoria antes de calcular as probabilidades. 
No contexto do projeto, ela é necessária para evitar que o bot calcule a probabilidade de uma palavra dada uma categoria como zero quando a palavra não aparece naquela categoria na base de treinamento.

Isso ocorre porque, sem a suavização, a probabilidade seria igual a zero, o que afetaria negativamente a classificação do bot. Ao adicionar uma constante 1, por exemplo, a todas as contagens de cada categoria, a suavização de LaPlace garante que as probabilidades calculadas pelo bot sejam mais confiáveis e precisas.

In [143]:
# Montagem do classificador com Suavização de LaPlace

def classificador(texto):
    clear = cleanup(texto.lower())
    split = clear.split()
    r1 = 1
    ir1 = 1
    n1 = 1
    pos = dfpos.value_counts()
    irrel = dfirrel.value_counts()
    neg = dfneg.value_counts()
    for word in split:
        if word in pos:
            r = pos[word]
        else:
            r = 0
        if word in irrel:
            ir = irrel[word]
        else:
            ir = 0
        if word in neg:
            n = neg[word]
        else:
            n = 0

#Suavização de Laplace
    r1 = r1*((r+1)/((len(dfpos))+len(dfall.value_counts())))
    ir1 = ir1*((ir+1)/((len(dfirrel))+len(dfall.value_counts())))
    n1 = n1*((n+1)/((len(dfneg))+ len(dfall.value_counts())))
    pp = r1*P
    pir = ir1*IR
    pn = n1*N

    l = [pp,pir,pn]
    if pp == max(l):
        return 1
    if pir == max(l):
        return 2
    if pn == max(l):
        return 0

    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

Com a função do classificador agora já definida, utiliza-se um laço **for** que percorra todas as frases na planilha de testes do Excel e construa uma lista com todas as categorias que o bot atribui a cada frase.

In [144]:
# Aplicação do classificador

i=0
l=[]

for review in test['Avaliacao']:
    aval = classificador(review)
    l.append(aval)
    i+=1
print(l)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 1, 2, 1, 1, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 2, 0, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 0, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 1, 1, 2, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 2, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 

Depois de imprimir a lista, basta jogar os dados obtidos em um **crosstab** com os dados originais para analisar os acertos do classificador.

In [145]:
# Analisando a precisão do classificador

pd.crosstab(test['Target'], l, normalize=True)*100

col_0,0,1,2
Target,,,
0,8.180301,23.038397,4.841402
1,4.841402,29.716194,3.839733
2,3.338898,13.021703,9.181970


Para entender quais foram os acertos, basta olhar a partir da posição (0,0): isso significa que tanto o Target quanto o classificador deram valor 0. Em (1,1), ambas atribuíram valor 1 e em (2,2), valor 2.

___
### Concluindo

O classificador funciona como proposto: o código está funcional e não há nenhum erro de repetição e de sintaxe. Ao anallisar, podemos perceber que o classificador é mais preciso para as avaliações positivas, mantendo a maior porcentagem (aproximadamente 29,7%). Porém, o mesmo não pode ser dito pelas avaliações negativas ou irrelevantes.

Ao analisar o código e as avaliações manualmente, percebemos que a maioria dos erros não se deve ao código **(possivelmente uma futura iteração com detalhamento maior no código poderia mitigar alguns problemas, mas não foi o analisado)**, mas sim às próprias avaliações dos usuários.

Isso se dá principalmente pois, de acordo com o critério de análise selecionado, **as avaliações irrelevantes são aquelas que não se referem ao livro, mas sim à entrega e qualidade do material físico, podendo ser tanto avaliações positivas ou negativas**. Por conta disso, algumas avaliações que originalmente eram irrelevantes **podem acabar assumindo o papel de avaliações positivas ou negativas**.

Um outro motivo, por fim, é um fora do controle do próprio bot: **a incapacidade de captar ironias e sátiras**. Algumas avaliações negativas utilizaram ironias e sátiras para dizer que o conteúdo do livro era ruim, assim prejudicando a análise do classificador. Por conta disso, **algumas avaliações negativas acabaram sendo positivas**.

Uma análise extra que pode ser considerada como um fator da imprecisão do bot é o fato de que **críticas e análises sem ditar uma avaliação com opinião foram consideradas irrelevantes**. Nestes casos, o bot pode ter assumido tais como avaliações positivas ou negativas. O mesmo se aplica para avaliações positivas e/ou negativas que possuíam críticas a certos aspectos do livro em questão.

Em uma futura iteração, o código poderia levar em conta padrões em algumas avaliações irrelevantes e nos casos irônicos para poder avaliar corretamente aquelas que foram avaliadas erroneamente. Além disso, poderiam ser implementadas algumas análises de pontuação para melhorar a análise do bot.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**